In [214]:
import pandas as pd 
pd.options.mode.chained_assignment = None
from pathlib import Path 
import numpy as np
from typing import Union 
from datetime import datetime, timedelta
from queue import PriorityQueue
import re
import geopy.distance
from timeit import default_timer as timer

DATA_DIR = Path('../data')

In [282]:
connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv', 
                               usecols=['line', 'departure_time', 'arrival_time', 'start_stop',
       'end_stop', 'start_stop_lat', 'start_stop_lon', 'end_stop_lat',
       'end_stop_lon'])

C:\Users\julia\AppData\Local\Temp\ipykernel_36184\2815982893.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',


In [283]:
connection_graph.dtypes

line               object
departure_time     object
arrival_time       object
start_stop         object
end_stop           object
start_stop_lat    float64
start_stop_lon    float64
end_stop_lat      float64
end_stop_lon      float64
dtype: object

In [217]:
connection_graph.columns

Index(['line', 'departure_time', 'arrival_time', 'start_stop', 'end_stop',
       'start_stop_lat', 'start_stop_lon', 'end_stop_lat', 'end_stop_lon'],
      dtype='object')

In [218]:
connection_graph.head()

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon
0,A,20:52:00,20:53:00,Zajezdnia Obornicka,Paprotna,51.148737,17.021069,51.147752,17.020539
1,A,20:53:00,20:54:00,Paprotna,Obornicka (Wołowska),51.147752,17.020539,51.144385,17.023735
2,A,20:54:00,20:55:00,Obornicka (Wołowska),Bezpieczna,51.144385,17.023735,51.141360,17.026376
3,A,20:55:00,20:57:00,Bezpieczna,Bałtycka,51.141360,17.026376,51.136632,17.030617
4,A,20:57:00,20:59:00,Bałtycka,Broniewskiego,51.136632,17.030617,51.135851,17.037383


In [284]:
len(connection_graph)

996520

In [285]:
connection_graph = connection_graph.drop_duplicates()

In [286]:
len(connection_graph)

456019

Obserwacja: niektóre linie są wyrażone jako int, a niektóre jako str - najlepiej to sprowadzić to wspólnego formatu str

In [222]:
connection_graph['line'].unique()

array(['A', 'D', 'K', 'N', '1', '2', 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
       13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 136, 137, 138, 142, 143, 144, 145, 146, 147,
       148, 149, 150, 151, 152, 206, 240, 241, 242, 243, 244, 245, 246,
       247, 248, 249, 250, 251, 253, 255, 257, 259, 310, 315, 319, 345,
       602, 607, 612, 714, 903, 904, 907, 908, 909, 911, 913, 914, 917,
       920, 921, 923, 924, 927, 930, 931, 933, 934, 936, 937, 938, 941,
       947, 948, 958, 967], dtype=object)

In [223]:
connection_graph['line'] = connection_graph['line'].astype(str)

Założenie: czas przejazdu między przystankami dla tej samej linii może być różny między różnymi godzinami 

In [224]:
krzyki_sowia_conn = connection_graph[(connection_graph['start_stop'] == 'KRZYKI') & (connection_graph['end_stop'] == 'Sowia')]

In [225]:
krzyki_sowia_conn['line'].unique()

array(['A', '126', '248', '251'], dtype=object)

In [226]:
duplicated_stops = connection_graph[connection_graph['start_stop'] == connection_graph['end_stop']]
duplicated_stops

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon
14127,D,08:43:00,08:44:00,Pomorska,Pomorska,51.117895,17.030893,51.117267,17.032001
15247,D,09:25:00,09:26:00,Pomorska,Pomorska,51.117895,17.030893,51.117267,17.032001
15934,D,20:49:00,20:50:00,Pomorska,Pomorska,51.117339,17.031928,51.118012,17.030745
16934,D,20:12:00,20:13:00,Pomorska,Pomorska,51.117339,17.031928,51.118012,17.030745
21374,D,05:46:00,05:47:00,Pomorska,Pomorska,51.117895,17.030893,51.117267,17.032001
...,...,...,...,...,...,...,...,...,...
995006,947,16:25:00,16:26:00,OPORÓW,OPORÓW,51.084150,16.971898,51.083929,16.970916
995033,947,18:25:00,18:26:00,OPORÓW,OPORÓW,51.084150,16.971898,51.083929,16.970916
995051,947,20:25:00,20:26:00,OPORÓW,OPORÓW,51.084150,16.971898,51.083929,16.970916
995078,947,05:05:00,05:06:00,OPORÓW,OPORÓW,51.084150,16.971898,51.083929,16.970916


POKAZAC ZE DLA TAKIEGO PRZYPADKU BEDZIE DZIALALO

W przypadku implementacji Dijkstry to, że istnieją połączenia między tym samym przystankiem nie będzie miało znaczenia, tj. nie będziemy takich połączeń rozważać

In [227]:
connection_graph.loc[duplicated_stops.iloc[0].name - 5:duplicated_stops.iloc[0].name + 5]

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon
14122,D,08:34:00,08:35:00,Obornicka (Wołowska),Bezpieczna,51.144385,17.023735,51.141360,17.026376
14123,D,08:35:00,08:36:00,Bezpieczna,Bałtycka,51.141360,17.026376,51.137180,17.029450
14124,D,08:36:00,08:39:00,Bałtycka,Kleczkowska,51.137180,17.029450,51.126714,17.027626
14125,D,08:39:00,08:40:00,Kleczkowska,pl. Strzelecki,51.126714,17.027626,51.122708,17.026304
14126,D,08:40:00,08:43:00,pl. Strzelecki,Pomorska,51.122708,17.026304,51.117895,17.030893
14127,D,08:43:00,08:44:00,Pomorska,Pomorska,51.117895,17.030893,51.117267,17.032001
14128,D,08:44:00,08:45:00,Pomorska,Mosty Pomorskie,51.117267,17.032001,51.114754,17.030627
14129,D,08:45:00,08:47:00,Mosty Pomorskie,Rynek,51.114754,17.030627,51.111912,17.027184
14130,D,08:47:00,08:49:00,Rynek,Narodowe Forum Muzyki,51.111912,17.027184,51.107899,17.026294
14131,D,08:49:00,08:51:00,Narodowe Forum Muzyki,Renoma,51.107899,17.026294,51.104286,17.030093


Heurystyka będzie brała pod uwagę odległość w metrach między przystankami. 

Czy trzeba koniecznie tą odległość manhattan / euklidesową?

In [228]:
stop1 = connection_graph.loc[995006]
geopy.distance.geodesic((stop1['start_stop_lat'], stop1['start_stop_lon']), 
                        (stop1['end_stop_lat'], stop1['end_stop_lon'])).m

73.09182397423561

In [229]:
stop1 = connection_graph.loc[14127]

In [230]:
def get_stop_coords(stop: str):
    return connection_graph[connection_graph['end_stop'] == stop].loc[:, ['end_stop', 'end_stop_lat', 'end_stop_lon']].drop_duplicates()

def compute_stop_coords(stop: str):
    stops = get_stop_coords(stop)
    return stops[['end_stop_lat', 'end_stop_lon']].mean()

In [231]:
get_stop_coords('Pomorska')

,end_stop,end_stop_lat,end_stop_lon
14126,Pomorska,51.117895,17.030893
14127,Pomorska,51.117267,17.032001
15933,Pomorska,51.117339,17.031928
15934,Pomorska,51.118012,17.030745
105438,Pomorska,51.117892,17.031002
239675,Pomorska,51.118058,17.030560
300020,Pomorska,51.117267,17.031899
300086,Pomorska,51.117234,17.031756


In [232]:
goal_stop_coords = compute_stop_coords('Pomorska')

A geodesic is the shortest path between two points on a curved surface, analogous to a straight line on a plane surface

In [390]:
def heuristic(curr_conn, goal_stop_coords):
    return round(geopy.distance.geodesic((curr_conn['end_stop_lat'], curr_conn['end_stop_lon']), 
                                   (goal_stop_coords['end_stop_lat'], goal_stop_coords['end_stop_lon'])).m, 2) 

heuristic(connection_graph.loc[14122], goal_stop_coords)

2663.87

Problem dotyczący dat:
- format nie jest jednolity, tj. istnieją godziny 25
- porównywanie godzin nie daje żądanych wyników, tj. chcielibyśmy, by godzina 00:00 była po 23:59

In [234]:
(datetime.strptime('00:00:00', '%H:%M:%S') > datetime.strptime('23:59:00', '%H:%M:%S'))

False

In [235]:
def separate_time(time):
    return [int(tp) for tp in time.split(':')]

In [236]:
separate_time('20:00:00')

[20, 0, 0]

In [237]:
connection_graph[connection_graph['departure_time'].apply(lambda x: separate_time(x)[0]) >= 24].head()

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon
30969,K,24:01:00,24:03:00,Broniewskiego,Bałtycka,51.135904,17.035327,51.138282,17.029374
30970,K,24:03:00,24:04:00,Bałtycka,Bezpieczna,51.138282,17.029374,51.142459,17.025678
30971,K,24:04:00,24:05:00,Bezpieczna,Paprotna,51.142459,17.025678,51.148181,17.020627
30972,K,24:05:00,24:06:00,Paprotna,Zajezdnia Obornicka,51.148181,17.020627,51.148737,17.021069
31418,K,24:00:00,24:03:00,DWORZEC GŁÓWNY,GALERIA DOMINIKAŃSKA,51.099602,17.035400,51.108019,17.037383


In [238]:
connection_graph[connection_graph['departure_time'].apply(lambda x: separate_time(x)[0]) >= 24].max()

line                      K
departure_time     30:03:00
arrival_time       30:05:00
start_stop          Żmudzka
end_stop            Żmudzka
start_stop_lat    51.196642
start_stop_lon    17.165968
end_stop_lat         51.197
end_stop_lon      17.165968
dtype: object

In [239]:
connection_graph[connection_graph['departure_time'].apply(lambda x: separate_time(x)[0]) <= 4].min()

line                      1
departure_time     03:34:00
arrival_time       03:36:00
start_stop           8 Maja
end_stop             8 Maja
start_stop_lat    50.980426
start_stop_lon    16.704534
end_stop_lat      50.984631
end_stop_lon      16.696798
dtype: object

1. Czas zostanie znormalizowany tak, by wszystkie godziny były w przedziale [0:00; 23:59]

In [240]:
def to_seconds(time: str) -> int:
    h, m, s = separate_time(time)
    return int(h * 3600 + m * 60 + s)

def time_to_normalized_sec(time: str) -> int:
    return to_seconds(time) % (3600 * 24)

In [241]:
connection_graph['departure_sec'] = connection_graph['departure_time'].apply(time_to_normalized_sec)
connection_graph['arrival_sec'] = connection_graph['arrival_time'].apply(time_to_normalized_sec)

In [242]:
connection_graph['arrival_sec']

0         75180
1         75240
2         75300
3         75420
4         75540
          ...  
996209    67140
996210    67260
996211    67320
996212    67380
996213    67500
Name: arrival_sec, Length: 456019, dtype: int64

In [243]:
print(connection_graph['departure_sec'].max() <= (24 * 3600))
print(connection_graph['arrival_sec'].max() <= (24 * 3600))

True
True


2. Stworzymy funkcję, która będzie respektować porządek godzinowy, nie liczbowy

In [244]:
def diff(ts: Union[pd.Series, int], td: int) -> int:
    '''Function that returns difference between ts and td times expressed as seconds 
        - note that this will never be a negative value'''
    d = ts - td 
    if isinstance(ts, pd.Series):
        d[d < 0] += 24 * 3600
    elif d < 0:
        d += 24 * 3600
    return d 

In [245]:
t_01_00 = 1 * 3600
t_22_00 = 22 * 3600
t_23_00 = 23 * 3600
t_00_01 = 0 + 60
t_21_00 = 21 * 3600
t_12_00 = 12 * 3600

print('01:00 - 22:00 = ', timedelta(seconds=diff(t_01_00, t_22_00)))
print('01:00 - 12:00 = ', timedelta(seconds=diff(t_01_00, t_12_00)))
print('23:00 - 22:00 = ', timedelta(seconds=diff(t_23_00, t_22_00)))
print('00:01 - 22:00 = ', timedelta(seconds=diff(t_00_01, t_22_00)))
print('23:00 - 12:00 = ', timedelta(seconds=diff(t_23_00, t_12_00)))
print('12:00 - 22:00 = ', timedelta(seconds=diff(t_12_00, t_22_00)))


01:00 - 22:00 =  3:00:00
01:00 - 12:00 =  13:00:00
23:00 - 22:00 =  1:00:00
00:01 - 22:00 =  2:01:00
23:00 - 12:00 =  11:00:00
12:00 - 22:00 =  14:00:00


In [246]:
t_20_00 = to_seconds('20:00:00')
t_20_01 = to_seconds('20:01:00')
t_20_02 = to_seconds('20:02:00')
t_19_59 = to_seconds('19:59:00')
t_20_03 = to_seconds('20:03:00')

print('19:59:00 - 20:00:00 = ', timedelta(seconds=diff(t_19_59, t_20_00)))
print('20:01:00 - 20:00:00 = ', timedelta(seconds=diff(t_20_01, t_20_00)))
print('20:02:00 - 20:00:00 = ', timedelta(seconds=diff(t_20_02, t_20_00)))
print('20:03:00 - 20:00:00 = ', timedelta(seconds=diff(t_20_03, t_20_00)))

19:59:00 - 20:00:00 =  23:59:00
20:01:00 - 20:00:00 =  0:01:00
20:02:00 - 20:00:00 =  0:02:00
20:03:00 - 20:00:00 =  0:03:00


In [247]:
krzyki_sowia_conn = connection_graph[(connection_graph['start_stop'] == 'KRZYKI') & (connection_graph['end_stop'] == 'Sowia')]
krzyki_sowia_conn.head()

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,departure_sec,arrival_sec
44,A,22:19:00,22:20:00,KRZYKI,Sowia,51.074884,17.006569,51.073793,17.001845,80340,80400
169,A,22:00:00,22:01:00,KRZYKI,Sowia,51.074884,17.006569,51.073793,17.001845,79200,79260
241,A,06:38:00,06:39:00,KRZYKI,Sowia,51.074884,17.006569,51.073793,17.001845,23880,23940
305,A,09:08:00,09:09:00,KRZYKI,Sowia,51.074884,17.006569,51.073793,17.001845,32880,32940
369,A,11:25:00,11:26:00,KRZYKI,Sowia,51.074884,17.006569,51.073793,17.001845,41100,41160


Przykład: dostajemy prawidłowe, logiczne sortowanie wg. godzin odjazdu

In [248]:
sorted_by_dep = krzyki_sowia_conn.sort_values(by='departure_sec')

In [249]:
sorted_by_dep

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,departure_sec,arrival_sec
911433,248,24:13:00,24:14:00,KRZYKI,Sowia,51.075272,17.006664,51.073793,17.001845,780,840
916404,251,24:23:00,24:24:00,KRZYKI,Sowia,51.074884,17.006569,51.073793,17.001845,1380,1440
911275,248,25:13:00,25:14:00,KRZYKI,Sowia,51.075272,17.006664,51.073793,17.001845,4380,4440
916115,251,25:23:00,25:24:00,KRZYKI,Sowia,51.074884,17.006569,51.073793,17.001845,4980,5040
911504,248,26:13:00,26:14:00,KRZYKI,Sowia,51.075272,17.006664,51.073793,17.001845,7980,8040
...,...,...,...,...,...,...,...,...,...,...,...
641864,126,22:18:00,22:19:00,KRZYKI,Sowia,51.075272,17.006664,51.073793,17.001845,80280,80340
44,A,22:19:00,22:20:00,KRZYKI,Sowia,51.074884,17.006569,51.073793,17.001845,80340,80400
2047,A,22:34:00,22:35:00,KRZYKI,Sowia,51.074884,17.006569,51.073793,17.001845,81240,81300
644431,126,22:46:00,22:47:00,KRZYKI,Sowia,51.075272,17.006664,51.073793,17.001845,81960,82020


Warto zauważyć, co może jest oczywsite, że do danego przystanku możemy się dostać wieloma liniami. Będzie to miało znaczenie takie, że mając do wyboru wiele linii do następnego przystanku możemy wybrać tą, którą najszybciej się dostaniemy (tj. arrival_time będzie najmniejszy), ponieważ zawsze możemy się przesiąść - więc nawet jeśli wybierzemy linię, która nie prowadzi nas do celu bezpośrednio, to możemy się na dowolnym przystanku przesiąść w prawidłową linię. 

In [250]:
connection_graph[connection_graph['start_stop'] == 'KRZYKI'][['line', 'end_stop']].drop_duplicates()

,line,end_stop
44,A,Sowia
14074,D,Zimowa
14153,D,Orla
64771,2,Orla
127704,6,Orla
139301,7,Radio i Telewizja
139311,7,Zajezdnia BOREK
139375,7,Orla
246676,14,Orla
283710,17,Radio i Telewizja


In [251]:
end_stops = connection_graph[(connection_graph['start_stop'] == 'KRZYKI')]['end_stop'].unique()

In [252]:
from_sliczna_143 = connection_graph[(connection_graph['start_stop'] == 'Śliczna') & (connection_graph['end_stop'] == 'Uniwersytet Ekonomiczny') & (connection_graph['line'] == '143')]

In [253]:
from_sliczna_143.sort_values(by='departure_sec')

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,departure_sec,arrival_sec
802911,143,05:12:00,05:15:00,Śliczna,Uniwersytet Ekonomiczny,51.085705,17.035382,51.089768,17.026732,18720,18900
795984,143,05:13:00,05:16:00,Śliczna,Uniwersytet Ekonomiczny,51.085705,17.035382,51.089768,17.026732,18780,18960
805822,143,05:34:00,05:37:00,Śliczna,Uniwersytet Ekonomiczny,51.085705,17.035382,51.089768,17.026732,20040,20220
797068,143,05:43:00,05:46:00,Śliczna,Uniwersytet Ekonomiczny,51.085705,17.035382,51.089768,17.026732,20580,20760
802361,143,05:52:00,05:55:00,Śliczna,Uniwersytet Ekonomiczny,51.085705,17.035382,51.089768,17.026732,21120,21300
...,...,...,...,...,...,...,...,...,...,...,...
795308,143,21:05:00,21:08:00,Śliczna,Uniwersytet Ekonomiczny,51.085705,17.035382,51.089768,17.026732,75900,76080
795907,143,21:35:00,21:38:00,Śliczna,Uniwersytet Ekonomiczny,51.085705,17.035382,51.089768,17.026732,77700,77880
796912,143,22:05:00,22:08:00,Śliczna,Uniwersytet Ekonomiczny,51.085705,17.035382,51.089768,17.026732,79500,79680
798266,143,22:35:00,22:38:00,Śliczna,Uniwersytet Ekonomiczny,51.085705,17.035382,51.089768,17.026732,81300,81480


In [254]:
type(end_stops)

numpy.ndarray

In [367]:
class Graph:

    def __init__(self, conn_graph, change_time_compute=None):
        self.conn_graph = conn_graph
        self.change_time_compute = change_time_compute
        self.transform()

    def transform(self):
        '''Applies time transformation functions to departure and arrival times - the results are available in new columns'''
        self.conn_graph = self.conn_graph.drop_duplicates()
        self.conn_graph.loc[:, 'line'] = self.conn_graph.loc[:, 'line'].astype(str)
        self.conn_graph.loc[:, 'departure_sec'] = self.conn_graph['departure_time'].apply(time_to_normalized_sec)
        self.conn_graph.loc[:, 'arrival_sec'] = self.conn_graph['arrival_time'].apply(time_to_normalized_sec)

    def get_start_idx(self):
        return -1

    def add_start_conn(self, dep_time: int, start_stop: str):
        coords = self.compute_stop_coords(start_stop)
        self.conn_graph.loc[self.get_start_idx()] = pd.Series({'end_stop': start_stop, 'departure_sec': dep_time, 'arrival_sec': dep_time,
                                                               'end_stop_lat': coords['end_stop_lat'], 'end_stop_lon': coords['end_stop_lon']})

    def remove_start_conn(self):
        self.conn_graph.drop(self.get_start_idx(), inplace=True)

    def get_stop_coords(self, stop: str):
        return self.conn_graph[self.conn_graph['end_stop'] == stop].loc[:, ['end_stop', 'end_stop_lat', 'end_stop_lon']].drop_duplicates()

    def compute_stop_coords(self, stop: str):
        stops = self.get_stop_coords(stop)
        return stops[['end_stop_lat', 'end_stop_lon']].mean()

    def get_nodes(self):
        '''should we also add end_stop?'''
        return self.conn_graph['start_stop'].unique()
    
    def get_lines(self, start_stop, end_stop):
        '''Returns lines connecting two stops'''
        return self.conn_graph[(self.conn_graph['start_stop'] == start_stop) 
                               & (self.conn_graph['end_stop'] == end_stop)]['line'].unique()
    
    def get_stops(self, line):
        '''? this may not make sense because sometimes the routes are different'''
        return self.conn_graph[self.conn_graph['line'] == line]['start_stop'].unique()

    def get_neighbour_stops(self, start_stop: str) -> np.array:
        '''Returns neighbouring end stops'''
        # return self.conn_graph[self.conn_graph['start_stop'] == start_stop][['line', 'end_stop']].drop_duplicates()
        return self.conn_graph[self.conn_graph['start_stop'] == start_stop]['end_stop'].unique()

    def get_conn_valid_time_arrivals(self, dep_time: int, start_stop: str, end_stop: str) -> pd.Series:
        conn = self.conn_graph[(self.conn_graph['start_stop'] == start_stop) 
                               & (self.conn_graph['end_stop'] == end_stop)]
 
        time_arrv_diff = diff(conn['arrival_sec'], dep_time)

        # Here we should add some optional time for changing lines, but we would also need to
        # pass the current line
        time_dep_diff = diff(conn['departure_sec'], dep_time)
        # Sort by the departure time, and for the same departure time favour the connection 
        # with the earliest arrival
        differences = (time_arrv_diff - time_dep_diff) >= 0
        valid_time_arrv_diff = time_arrv_diff[differences]

        return conn, valid_time_arrv_diff

    def get_sorted_conns(self, dep_time: int, start_stop: str, end_stop: str):
        '''get all connections sorted by time arrival'''
        conn, valid_time_arrv_diff = self.get_conn_valid_time_arrivals(dep_time, start_stop, end_stop)

        return conn.loc[valid_time_arrv_diff.sort_values().index]

    def get_earliest_conn(self, dep_time: int, start_stop: str, end_stop: str) -> pd.Series:
        '''Returns the earliest connection between two stops'''
        conn, valid_time_arrv_diff = self.get_conn_valid_time_arrivals(dep_time, start_stop, end_stop)

        return conn.loc[valid_time_arrv_diff.idxmin()]
    
    def get_earliest_from(self, dep_time: int, start_stop: str):
        '''Returns all earliest connections to all neighbouring stops'''
        return [self.get_earliest_conn(dep_time, start_stop, end_stop) for 
                end_stop in self.get_neighbour_stops(start_stop)]

    def sub_cost(self, dep_time: int, next_idx: int, curr_idx: int = None) -> int:
        cost = diff(self.conn_graph.loc[next_idx, 'arrival_sec'], dep_time)
        # cost += self.change_time if current['line'] is not None and current['line'] != next['line'] else 0
        return cost
    
    def conn_at_index(self, index: int) -> pd.Series:
        return self.conn_graph.loc[index] 

In [368]:
g = Graph(connection_graph)

In [369]:
dep_time = 8 * 3600

In [370]:
g.add_start_conn(dep_time, 'PL. GRUNWALDZKI')

In [361]:
g.remove_start_conn()

In [366]:
g.compute_stop_coords('PL. GRUNWALDZKI')['end_stop_lat']

51.111746845624985

In [371]:
g.conn_at_index(-1)

line                          NaN
departure_time                NaN
arrival_time                  NaN
start_stop                    NaN
end_stop          PL. GRUNWALDZKI
start_stop_lat                NaN
start_stop_lon                NaN
end_stop_lat            51.111747
end_stop_lon            17.061436
departure_sec               28800
arrival_sec                 28800
Name: -1, dtype: object

In [260]:
g.conn_at_index(71821)

line                                              2
departure_time                             08:02:00
arrival_time                               08:04:00
start_stop                          PL. GRUNWALDZKI
end_stop          Kliniki - Politechnika Wrocławska
start_stop_lat                            51.111452
start_stop_lon                            17.060529
end_stop_lat                              51.109206
end_stop_lon                              17.066414
departure_sec                                 28920
arrival_sec                                   29040
Name: 71821, dtype: object

In [261]:
# cost for the first node
g.sub_cost(dep_time, 71821, -1)

240

In [262]:
g.get_neighbour_stops('PL. GRUNWALDZKI')

array(['most Grunwaldzki', 'Kochanowskiego',
       'Kliniki - Politechnika Wrocławska', 'Piastowska', 'Reja',
       'Bujwida', 'PL. GRUNWALDZKI'], dtype=object)

In [263]:
g.compute_stop_coords('PL. GRUNWALDZKI')

end_stop_lat    51.111747
end_stop_lon    17.061436
dtype: float64

In [264]:
g.get_neighbour_stops('Kliniki - Politechnika Wrocławska')

array(['Hala Stulecia', 'PL. GRUNWALDZKI', 'Mickiewicza'], dtype=object)

In [265]:
conn = connection_graph[(connection_graph['start_stop'] == 'PL. GRUNWALDZKI') & (connection_graph['end_stop'] == 'Kliniki - Politechnika Wrocławska')]

time_diff = diff(conn['departure_sec'], dep_time)
conn.loc[time_diff.sort_values().index].head(10)

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,departure_sec,arrival_sec
551172,115,08:00:00,08:02:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,51.111438,17.060353,51.109283,17.066162,28800,28920
325694,19,08:01:00,08:03:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,51.111141,17.061193,51.109206,17.066414,28860,28980
54062,1,08:01:00,08:02:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,51.111141,17.061193,51.109206,17.066414,28860,28920
71821,2,08:02:00,08:04:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,51.111452,17.060529,51.109206,17.066414,28920,29040
320086,19,08:03:00,08:04:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,51.111141,17.061193,51.109206,17.066414,28980,29040
543863,115,08:03:00,08:05:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,51.111438,17.060353,51.109283,17.066162,28980,29100
60334,1,08:04:00,08:06:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,51.111141,17.061193,51.109206,17.066414,29040,29160
99119,4,08:06:00,08:08:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,51.111452,17.060529,51.109206,17.066414,29160,29280
851072,146,08:07:00,08:09:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,51.111438,17.060353,51.109283,17.066162,29220,29340
191231,10,08:07:00,08:09:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,51.111452,17.060529,51.109206,17.066414,29220,29340


In [266]:
time_arrv_diff = diff(conn['arrival_sec'], dep_time)
time_dep_diff = diff(conn['departure_sec'], dep_time)

In [267]:
# We want both arrival and departure time to be one the 'same' side - that is, they are either after the 
# given dept_time or before
differences = (time_arrv_diff - time_dep_diff) >= 0

In [268]:
valid_time_arrv_diff = time_arrv_diff[differences]
conn.loc[valid_time_arrv_diff.idxmin()]

line                                              1
departure_time                             08:01:00
arrival_time                               08:02:00
start_stop                          PL. GRUNWALDZKI
end_stop          Kliniki - Politechnika Wrocławska
start_stop_lat                            51.111141
start_stop_lon                            17.061193
end_stop_lat                              51.109206
end_stop_lon                              17.066414
departure_sec                                 28860
arrival_sec                                   28920
Name: 54062, dtype: object

In [269]:
pd.concat([conn['departure_time'], conn['arrival_time'], time_dep_diff, time_arrv_diff, differences], axis=1).loc[time_arrv_diff.sort_values().index]

,departure_time,arrival_time,departure_sec,arrival_sec,0
104876,07:59:00,08:01:00,86340,60,False
840500,07:59:00,08:01:00,86340,60,False
54062,08:01:00,08:02:00,60,120,True
551172,08:00:00,08:02:00,0,120,True
325694,08:01:00,08:03:00,60,180,True
...,...,...,...,...,...
60250,07:52:00,07:54:00,85920,86040,True
850109,07:52:00,07:54:00,85920,86040,True
845919,07:55:00,07:57:00,86100,86220,True
193979,07:55:00,07:57:00,86100,86220,True


In [270]:
conn[differences].loc[time_arrv_diff[differences].sort_values().index]

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,departure_sec,arrival_sec
54062,1,08:01:00,08:02:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,51.111141,17.061193,51.109206,17.066414,28860,28920
551172,115,08:00:00,08:02:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,51.111438,17.060353,51.109283,17.066162,28800,28920
325694,19,08:01:00,08:03:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,51.111141,17.061193,51.109206,17.066414,28860,28980
71821,2,08:02:00,08:04:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,51.111452,17.060529,51.109206,17.066414,28920,29040
320086,19,08:03:00,08:04:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,51.111141,17.061193,51.109206,17.066414,28980,29040
...,...,...,...,...,...,...,...,...,...,...,...
850109,146,07:52:00,07:54:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,51.111438,17.060353,51.109283,17.066162,28320,28440
60250,1,07:52:00,07:54:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,51.111141,17.061193,51.109206,17.066414,28320,28440
845919,146,07:55:00,07:57:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,51.111438,17.060353,51.109283,17.066162,28500,28620
193979,10,07:55:00,07:57:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,51.111452,17.060529,51.109206,17.066414,28500,28620


In [271]:
g.get_earliest_conn(dep_time, 'PL. GRUNWALDZKI', 'Kliniki - Politechnika Wrocławska')

line                                              1
departure_time                             08:01:00
arrival_time                               08:02:00
start_stop                          PL. GRUNWALDZKI
end_stop          Kliniki - Politechnika Wrocławska
start_stop_lat                            51.111141
start_stop_lon                            17.061193
end_stop_lat                              51.109206
end_stop_lon                              17.066414
departure_sec                                 28860
arrival_sec                                   28920
Name: 54062, dtype: object

In [272]:
g.get_sorted_conns(dep_time, 'PL. GRUNWALDZKI', 'Kliniki - Politechnika Wrocławska')

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,departure_sec,arrival_sec
54062,1,08:01:00,08:02:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,51.111141,17.061193,51.109206,17.066414,28860,28920
551172,115,08:00:00,08:02:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,51.111438,17.060353,51.109283,17.066162,28800,28920
325694,19,08:01:00,08:03:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,51.111141,17.061193,51.109206,17.066414,28860,28980
320086,19,08:03:00,08:04:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,51.111141,17.061193,51.109206,17.066414,28980,29040
71821,2,08:02:00,08:04:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,51.111452,17.060529,51.109206,17.066414,28920,29040
...,...,...,...,...,...,...,...,...,...,...,...
60250,1,07:52:00,07:54:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,51.111141,17.061193,51.109206,17.066414,28320,28440
850109,146,07:52:00,07:54:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,51.111438,17.060353,51.109283,17.066162,28320,28440
193979,10,07:55:00,07:57:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,51.111452,17.060529,51.109206,17.066414,28500,28620
845919,146,07:55:00,07:57:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,51.111438,17.060353,51.109283,17.066162,28500,28620


In [273]:
g.conn_graph.loc[71821:71822]

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,departure_sec,arrival_sec
71821,2,08:02:00,08:04:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,51.111452,17.060529,51.109206,17.066414,28920,29040
71822,2,08:04:00,08:06:00,Kliniki - Politechnika Wrocławska,Hala Stulecia,51.109206,17.066414,51.106895,17.073720,29040,29160


Od 8:00 dostaniemy się o 8:04, więc czas przejazdu - czyli koszt - to 240 sekund

In [274]:
goal_stop_coors = g.compute_stop_coords('Tramwajowa')

In [275]:
g.sub_cost(dep_time, 71821) + heuristic(g.conn_graph.loc[71821], goal_stop_coors)

1583.87

In [276]:
g.get_earliest_from(dep_time, 'Kliniki - Politechnika Wrocławska')

[line                                              4
 departure_time                             08:01:00
 arrival_time                               08:03:00
 start_stop        Kliniki - Politechnika Wrocławska
 end_stop                              Hala Stulecia
 start_stop_lat                            51.109206
 start_stop_lon                            17.066414
 end_stop_lat                              51.106895
 end_stop_lon                               17.07372
 departure_sec                                 28860
 arrival_sec                                   28980
 Name: 104877, dtype: object,
 line                                              2
 departure_time                             08:00:00
 arrival_time                               08:02:00
 start_stop        Kliniki - Politechnika Wrocławska
 end_stop                            PL. GRUNWALDZKI
 start_stop_lat                            51.109713
 start_stop_lon                            17.065111
 end_stop_lat   

In [277]:
frontier = PriorityQueue()
frontier.put((10, 1))
frontier.put((5, 3))
print(frontier.get())
frontier.put((5, 3))
frontier.put((2, 8))
print(frontier.get())

(5, 3)
(2, 8)


In [278]:
frontier.get()

(5, 3)

Krotka jest w formacie (priority, data)

In [372]:
def print_info(conn, file=None):
    print(f"{conn['line']} goes from {conn['start_stop']} to {conn['end_stop']} and leaves at {conn['departure_time']}, arrives at {conn['arrival_time']}", 
          file=file)

In [373]:
def sec_to_time(seconds: int) -> str:
    seconds = int(seconds)
    hour = (seconds // 3600)
    minutes = (seconds % 3600) // 60 
    secs = (seconds % 3600) % 60 
    return f'{hour:02d}:{minutes:02d}:{secs:02d}'

In [374]:
def path_to_list(node: str, connections: dict):
    path = [node]
    while node:
        node = connections[node]
        path.append(node)
    path.reverse()
    return path[1:]

def idxs_to_nodes(graph: Graph, goal_idx: int, conn_idxs: dict):
    idx_path = path_to_list(goal_idx, conn_idxs)
    return [graph.conn_at_index(idx) for idx in idx_path[1:]]

def print_path(connections: dict):
    for conn in connections:
        print(f'{conn["start_stop"]} [{conn["departure_time"]}] --- {conn["line"]} ---> {conn["end_stop"]} [{conn["arrival_time"]}]')

In [375]:
def run_solution(find_path_function, start_stop: str, goal_stop: str, leave_hour: str, change_time=0):
    start = timer()
    connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv', 
                               usecols=['line', 'departure_time', 'arrival_time', 'start_stop',
       'end_stop', 'start_stop_lat', 'start_stop_lon', 'end_stop_lat', 'end_stop_lon'])
    graph = Graph(connection_graph, change_time)
    goal_index, came_from, costs = find_path_function(graph, start_stop, goal_stop, leave_hour)
    end = timer()
    elapsed_time = (end - start)
    solution_cost = costs[goal_stop]
    return graph, goal_index, came_from, solution_cost, elapsed_time 

In [376]:
test_cases = pd.read_json(DATA_DIR / 'test_cases.json')
test_cases['dept_time'] = test_cases['dept_time'].apply(lambda x: x.strftime('%H:%M:%S'))

test_cases = test_cases.values

In [385]:
def a_star_find_path(graph: Graph, start_stop: str, goal_stop: str, leave_hour: str):
   
    dep_time = time_to_normalized_sec(leave_hour)
    graph.add_start_conn(dep_time, start_stop)

    goal_stop_coords = graph.compute_stop_coords(goal_stop)

    cost = {}
    # if commuting A -> B, then this will be came_from[B] = A so we can recreate the path
    came_from = {}
    came_from[graph.get_start_idx()] = None
    cost[start_stop] = 0

    commute_so_far = {}
    commute_so_far[start_stop] = 0

    opened = [graph.get_start_idx()]
    closed = [] 

    def update(next_conn, current):
        next = next_conn['end_stop']
        commute_so_far[next] = commute_so_far[current] + graph.sub_cost(dep_time + commute_so_far[current], next_conn.name)
        cost[next] = commute_so_far[next] + heuristic(next_conn, goal_stop_coords)
        came_from[next_conn.name] = curr_conn.name

    with open(DATA_DIR / ('a_star_runs/' + re.sub(r"\W+", "", start_stop) + '-' + re.sub(r"\W+", "", goal_stop) + '.txt'), mode='w', encoding='utf-8') as f:
        
        i = 0
        while len(opened) > 0:
            print(f'[{i}]', file=f)
            curr_idx = None 
            curr_cost = None 

            # choose node with the lowest cost: what if we choose a connection 
            # that ends at the particular stop?
            for conn_idx in opened:
                candidate_stop = graph.conn_at_index(conn_idx)['end_stop']
                if curr_idx is None or cost[candidate_stop] < curr_cost:
                    curr_idx = conn_idx 
                    curr_cost = cost[candidate_stop]
                    
            curr_conn = graph.conn_at_index(curr_idx)
            current = curr_conn['end_stop']
            if current == goal_stop:
                goal_index = curr_conn.name 
                break 

            opened.remove(curr_idx)
            closed.append(curr_idx)
            
            for next_conn in graph.get_earliest_from(dep_time=dep_time + commute_so_far[current], start_stop=current):
                print_info(next_conn, file=f)
                next = next_conn['end_stop']
                if next_conn.name not in opened and next_conn.name not in closed:
                    opened.append(next_conn.name)
                    update(next_conn, current)
                    print(f"[NEW] We arrive from {start_stop} to {next} at {sec_to_time(dep_time + commute_so_far[next])}", file=f)
                else:
                    if commute_so_far[next] > commute_so_far[current] + graph.sub_cost(dep_time + commute_so_far[current], next_conn.name):
                        update(next_conn, current)
                        print(f"[UPDATED] We arrive from {start_stop} to {next} at {sec_to_time(dep_time + commute_so_far[next])}", file=f)
                        if next_conn.name in closed:
                            closed.remove(next_conn.name)
                            opened.append(next_conn.name)
            print(f'OPENED = {opened}', file=f)
            print(f'CLOSED = {closed}', file=f)
            print(f'Commute so far = {commute_so_far}', file=f)
            print(f'Cost = {cost}', file=f)
            print(f'--------------[{i}]------------------', file=f)
            i += 1
    
    return goal_index, came_from, cost


In [378]:
graph, goal_index, came_from, solution_cost, elapsed_time = run_solution(a_star_find_path, 'Pomorska', 'Pomorska', '16:00:00')

C:\Users\julia\AppData\Local\Temp\ipykernel_36184\2581666429.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',


In [379]:
goal_index

-1

In [399]:
def a_star(start_stop: str, goal_stop: str, leave_hour: str, change_time=0):
    print(f'Commute from {start_stop} to {goal_stop} at {leave_hour}')
    graph, goal_index, came_from, solution_cost, elapsed_time = run_solution(a_star_find_path, start_stop, goal_stop, leave_hour, change_time)
    connections = idxs_to_nodes(graph, goal_index, came_from)
    print_path(connections)
    print(f'Solution cost is {solution_cost:.2f}')
    commuting_time = sec_to_time(diff(connections[-1]['arrival_sec'], connections[0]['departure_sec']))
    print(f"Total time is {sec_to_time(diff(connections[-1]['arrival_sec'], time_to_normalized_sec(leave_hour)))}")
    print(f'Commuting takes {commuting_time}')
    print(f'Algorithm took {elapsed_time:.2f}s to execute')

In [400]:
a_star('Pomorska', 'Pomorska', '16:00:00')

Commute from Pomorska to Pomorska at 16:00:00


C:\Users\julia\AppData\Local\Temp\ipykernel_36184\2581666429.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',


Solution cost is 0.00


IndexError: list index out of range

In [401]:
a_star(*test_cases[4])

Commute from PL. GRUNWALDZKI to Hala Stulecia at 08:00:00


C:\Users\julia\AppData\Local\Temp\ipykernel_36184\2581666429.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',


PL. GRUNWALDZKI [08:01:00] --- 1 ---> Kliniki - Politechnika Wrocławska [08:02:00]
Kliniki - Politechnika Wrocławska [08:02:00] --- 1 ---> Hala Stulecia [08:04:00]
Solution cost is 285.36
Total time is 00:04:00
Commuting takes 00:03:00
Algorithm took 6.11s to execute


In [402]:
a_star(*test_cases[0])

Commute from KLECINA to OSIEDLE SOBIESKIEGO at 20:00:00


C:\Users\julia\AppData\Local\Temp\ipykernel_36184\2581666429.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',


KLECINA [20:02:00] --- 7 ---> Sąsiedzka [20:03:00]
Sąsiedzka [20:03:00] --- 7 ---> Braterska [20:04:00]
Braterska [20:04:00] --- 7 ---> Przyjaźni [20:05:00]
Przyjaźni [20:36:00] --- 113 ---> KRZYKI [20:39:00]
KRZYKI [23:09:00] --- 602 ---> Hallera [23:12:00]
Hallera [20:17:00] --- 143 ---> Drukarska [20:20:00]
Drukarska [20:20:00] --- 143 ---> Uniwersytet Ekonomiczny [20:21:00]
Uniwersytet Ekonomiczny [20:26:00] --- 15 ---> Sanocka [20:27:00]
Sanocka [20:27:00] --- 15 ---> DWORZEC AUTOBUSOWY [20:29:00]
DWORZEC AUTOBUSOWY [20:29:00] --- 15 ---> Dworzec Główny (Stawowa) [20:30:00]
Dworzec Główny (Stawowa) [20:30:00] --- 15 ---> Arkady (Capitol) [20:32:00]
Arkady (Capitol) [20:54:00] --- D ---> GALERIA DOMINIKAŃSKA [20:59:00]
GALERIA DOMINIKAŃSKA [20:42:00] --- N ---> Urząd Wojewódzki (Muzeum Narodowe) [20:45:00]
Urząd Wojewódzki (Muzeum Narodowe) [20:45:00] --- N ---> Katedra [20:46:00]
Katedra [20:46:00] --- N ---> Ogród Botaniczny [20:48:00]
Ogród Botaniczny [20:48:00] --- N ---> Wyszy

In [403]:
a_star(*test_cases[1])

Commute from Broniewskiego to Uniwersytet Ekonomiczny at 10:15:00


C:\Users\julia\AppData\Local\Temp\ipykernel_36184\2581666429.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',


Broniewskiego [10:19:00] --- 7 ---> Trzebnicka [10:22:00]
Trzebnicka [10:22:00] --- 7 ---> DWORZEC NADODRZE [10:24:00]
DWORZEC NADODRZE [10:24:00] --- 7 ---> Paulińska [10:26:00]
Paulińska [10:26:00] --- 7 ---> Dubois [10:28:00]
Dubois [10:49:00] --- 132 ---> Rynek [10:53:00]
Rynek [10:53:00] --- K ---> Świdnicka [10:56:00]
Świdnicka [10:36:00] --- 10 ---> Park Staromiejski [10:38:00]
Park Staromiejski [10:38:00] --- 10 ---> Wzgórze Partyzantów [10:39:00]
Wzgórze Partyzantów [10:39:00] --- 9 ---> DWORZEC GŁÓWNY [10:41:00]
DWORZEC GŁÓWNY [10:42:00] --- 9 ---> DWORZEC AUTOBUSOWY [10:44:00]
DWORZEC AUTOBUSOWY [10:44:00] --- 9 ---> Sanocka [10:46:00]
Sanocka [10:46:00] --- 9 ---> Uniwersytet Ekonomiczny [10:47:00]
Solution cost is 1951.43
Total time is 00:32:00
Commuting takes 00:28:00
Algorithm took 9.03s to execute


In [406]:
a_star(*test_cases[2])

Commute from POŚWIĘTNE to Młodych Techników at 15:30:00


C:\Users\julia\AppData\Local\Temp\ipykernel_36184\2581666429.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',


POŚWIĘTNE [15:33:00] --- 15 ---> Wołowska [15:34:00]
Wołowska [15:34:00] --- 15 ---> Kępińska [15:35:00]
Kępińska [15:35:00] --- 15 ---> Kamieńskiego [15:36:00]
Kamieńskiego [15:36:00] --- 15 ---> Broniewskiego [15:38:00]
Broniewskiego [15:38:00] --- 15 ---> Trzebnicka [15:41:00]
Trzebnicka [15:41:00] --- 15 ---> DWORZEC NADODRZE [15:43:00]
DWORZEC NADODRZE [15:43:00] --- 15 ---> Paulińska [15:45:00]
Paulińska [15:45:00] --- 15 ---> Dubois [15:48:00]
Dubois [15:49:00] --- 132 ---> Rynek [15:53:00]
Rynek [15:53:00] --- 132 ---> PL. JANA PAWŁA II [15:55:00]
PL. JANA PAWŁA II [15:55:00] --- 3 ---> Młodych Techników [15:57:00]
Solution cost is 1636.66
Total time is 00:27:00
Commuting takes 00:24:00
Algorithm took 8.21s to execute


In [408]:
a_star(*test_cases[3])

Commute from Śliczna to Marchewkowa at 23:50:00


C:\Users\julia\AppData\Local\Temp\ipykernel_36184\2581666429.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',


MemoryError: 

In [392]:
def find_path(graph: Graph, start_stop: str, goal_stop: str, leave_hour: str):
    
    frontier = PriorityQueue()
    dep_time = time_to_normalized_sec(leave_hour)
    graph.add_start_conn(dep_time, start_stop)

    cost_so_far = {}
    # if commuting A -> B, then this will be came_from[B] = A so we can recreate the path
    came_from = {}
    cost_so_far[start_stop] = 0
    came_from[graph.get_start_idx()] = None

    frontier.put((cost_so_far[start_stop], graph.get_start_idx()))

    with open(DATA_DIR / ('dijkstra_runs/' + re.sub(r"\W+", "", start_stop) + '-' + re.sub(r"\W+", "", goal_stop)), mode='w', encoding='utf-8') as f:

        i = 0
        while not frontier.empty():
            # get the stop with the lowest cost 
            cost, index = frontier.get()

            conn = graph.conn_at_index(index)
            current = conn['end_stop']

            if current == goal_stop:
                goal_index = index
                break 

            print(f'[{i}]', file=f)
            for next_conn in graph.get_earliest_from(dep_time=dep_time + cost, start_stop=current):
                print_info(next_conn, file=f)
                next = next_conn['end_stop']
                # cost of commuting start --> current and current --> next 
                new_cost = cost + graph.sub_cost(dep_time + cost, next_conn.name, conn.name)
                if next not in cost_so_far or new_cost < cost_so_far[next]:
                    print(f'We arrive from {start_stop} to {next} at {sec_to_time(dep_time + new_cost)}', file=f)
                    cost_so_far[next] = new_cost
                    frontier.put((new_cost, next_conn.name))
                    came_from[next_conn.name] = index
            i+=1

    return goal_index, came_from, cost_so_far

In [393]:
def dijkstra(start_stop: str, goal_stop: str, leave_hour: str, change_time=0):
    print(f'Commute from {start_stop} to {goal_stop} at {leave_hour}')
    graph, goal_index, came_from, solution_cost, elapsed_time = run_solution(find_path, start_stop, goal_stop, leave_hour, change_time)
    connections = idxs_to_nodes(graph, goal_index, came_from)
    print_path(connections)
    print(f'Total trip time is {sec_to_time(solution_cost)}')
    commuting_time = sec_to_time(diff(connections[-1]['arrival_sec'], connections[0]['departure_sec']))
    print(f'Commuting takes {commuting_time}')
    print(f'Algorithm took {elapsed_time:.2f}s to execute')

Przypadki testowe:


In [394]:
dijkstra(*test_cases[0])

Commute from KLECINA to OSIEDLE SOBIESKIEGO at 20:00:00


C:\Users\julia\AppData\Local\Temp\ipykernel_36184\2581666429.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',


KLECINA [20:01:00] --- 107 ---> Skarbowców [20:02:00]
Skarbowców [20:02:00] --- 107 ---> Os. Przyjaźni [20:03:00]
Os. Przyjaźni [20:03:00] --- 107 ---> Zimowa [20:04:00]
Zimowa [20:04:00] --- 107 ---> KRZYKI [20:05:00]
KRZYKI [20:05:00] --- 612 ---> PARK POŁUDNIOWY [20:07:00]
PARK POŁUDNIOWY [20:08:00] --- 9 ---> Pułtuska [20:09:00]
Pułtuska [20:09:00] --- 9 ---> Weigla (Szpital) [20:10:00]
Weigla (Szpital) [20:10:00] --- 9 ---> Jaworowa [20:11:00]
Jaworowa [20:11:00] --- 9 ---> Wiśniowa [20:12:00]
Wiśniowa [20:12:00] --- 9 ---> Uniwersytet Ekonomiczny [20:14:00]
Uniwersytet Ekonomiczny [20:14:00] --- 9 ---> Sanocka [20:15:00]
Sanocka [20:15:00] --- 9 ---> DWORZEC AUTOBUSOWY [20:17:00]
DWORZEC AUTOBUSOWY [20:17:00] --- 9 ---> DWORZEC GŁÓWNY [20:19:00]
DWORZEC GŁÓWNY [20:20:00] --- K ---> GALERIA DOMINIKAŃSKA [20:24:00]
GALERIA DOMINIKAŃSKA [20:24:00] --- 9 ---> pl. Nowy Targ [20:25:00]
pl. Nowy Targ [20:25:00] --- 9 ---> Hala Targowa [20:26:00]
Hala Targowa [20:26:00] --- 9 ---> pl. Be

In [395]:
dijkstra(*test_cases[1])

Commute from Broniewskiego to Uniwersytet Ekonomiczny at 10:15:00


C:\Users\julia\AppData\Local\Temp\ipykernel_36184\2581666429.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',


Broniewskiego [10:16:00] --- 111 ---> Zakładowa [10:18:00]
Zakładowa [10:18:00] --- 111 ---> Słonimskiego [10:20:00]
Słonimskiego [10:20:00] --- 111 ---> Daszyńskiego [10:23:00]
Daszyńskiego [10:23:00] --- 111 ---> Nowowiejska [10:25:00]
Nowowiejska [10:25:00] --- 111 ---> Jedności Narodowej [10:26:00]
Jedności Narodowej [10:26:00] --- 111 ---> Na Szańcach [10:27:00]
Na Szańcach [10:28:00] --- 8 ---> pl. Bema [10:30:00]
pl. Bema [10:30:00] --- 8 ---> Hala Targowa [10:32:00]
Hala Targowa [10:32:00] --- 8 ---> pl. Nowy Targ [10:33:00]
pl. Nowy Targ [10:33:00] --- 8 ---> GALERIA DOMINIKAŃSKA [10:35:00]
GALERIA DOMINIKAŃSKA [10:36:00] --- 5 ---> Wzgórze Partyzantów [10:37:00]
Wzgórze Partyzantów [10:37:00] --- 5 ---> DWORZEC GŁÓWNY [10:39:00]
DWORZEC GŁÓWNY [10:40:00] --- 8 ---> DWORZEC AUTOBUSOWY [10:43:00]
DWORZEC AUTOBUSOWY [10:44:00] --- 9 ---> Sanocka [10:46:00]
Sanocka [10:46:00] --- 9 ---> Uniwersytet Ekonomiczny [10:47:00]
Total trip time is 00:32:00
Commuting takes 00:31:00
Algori

In [404]:
dijkstra(*test_cases[2])

Commute from POŚWIĘTNE to Młodych Techników at 15:30:00


C:\Users\julia\AppData\Local\Temp\ipykernel_36184\2581666429.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',


POŚWIĘTNE [15:33:00] --- 15 ---> Wołowska [15:34:00]
Wołowska [15:34:00] --- 15 ---> Kępińska [15:35:00]
Kępińska [15:35:00] --- 15 ---> Kamieńskiego [15:36:00]
Kamieńskiego [15:36:00] --- 15 ---> Broniewskiego [15:38:00]
Broniewskiego [15:38:00] --- 15 ---> Trzebnicka [15:41:00]
Trzebnicka [15:41:00] --- 15 ---> DWORZEC NADODRZE [15:43:00]
DWORZEC NADODRZE [15:43:00] --- 15 ---> Paulińska [15:45:00]
Paulińska [15:45:00] --- 15 ---> Dubois [15:48:00]
Dubois [15:48:00] --- 15 ---> Pomorska [15:50:00]
Pomorska [15:50:00] --- 15 ---> Kępa Mieszczańska [15:53:00]
Kępa Mieszczańska [15:53:00] --- 144 ---> PL. JANA PAWŁA II [15:55:00]
PL. JANA PAWŁA II [15:55:00] --- 3 ---> Młodych Techników [15:57:00]
Total trip time is 00:27:00
Commuting takes 00:24:00
Algorithm took 24.71s to execute


Zmien czas by dawal czas przejazdu bez czekania na poczatku

In [405]:
dijkstra(*test_cases[3])

Commute from Śliczna to Marchewkowa at 23:50:00


C:\Users\julia\AppData\Local\Temp\ipykernel_36184\2581666429.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',


Śliczna [23:55:00] --- K ---> Borowska (Aquapark) [23:56:00]
Borowska (Aquapark) [23:56:00] --- K ---> DWORZEC AUTOBUSOWY [23:58:00]
DWORZEC AUTOBUSOWY [23:58:00] --- K ---> DWORZEC GŁÓWNY [24:00:00]
DWORZEC GŁÓWNY [24:02:00] --- 23 ---> Arkady (Capitol) [24:05:00]
Arkady (Capitol) [24:02:00] --- 250 ---> EPI [24:04:00]
EPI [24:05:00] --- 248 ---> Zaolziańska [24:06:00]
Zaolziańska [24:01:00] --- 23 ---> Wielka [24:02:00]
Wielka [24:00:00] --- 21 ---> Rondo [24:01:00]
Rondo [24:01:00] --- 21 ---> Sztabowa [24:02:00]
Sztabowa [24:00:00] --- 7 ---> Hallera [24:01:00]
Hallera [24:00:00] --- 2 ---> Jastrzębia [24:01:00]
Jastrzębia [24:01:00] --- 2 ---> Orla [24:02:00]
Orla [24:03:00] --- 7 ---> KRZYKI [24:06:00]
KRZYKI [24:06:00] --- 7 ---> Radio i Telewizja [24:07:00]
Radio i Telewizja [24:07:00] --- 7 ---> Przyjaźni [24:09:00]
Przyjaźni [24:09:00] --- 7 ---> Braterska [24:10:00]
Braterska [24:10:00] --- 7 ---> Sąsiedzka [24:11:00]
Sąsiedzka [24:11:00] --- 7 ---> KLECINA [24:12:00]
KLECIN

In [605]:
graph, goal_index, came_from, solution_cost, elapsed_time = run_solution(*test_cases[3])

In [606]:
came_from

{-1: None,
 12394: -1,
 254766: -1,
 256893: -1,
 226197: -1,
 256987: 254766,
 212092: 254766,
 254767: 254766,
 250184: 254767,
 252636: 254767,
 257603: 254767,
 85400: 226197,
 257605: 226197,
 226198: 226197,
 212120: 226198,
 226199: 226198,
 182642: 226199,
 180665: 226199,
 254396: 226199,
 254041: 226199,
 257428: 226199,
 226200: 226199,
 215921: 226199,
 261345: 226199,
 261427: 226199,
 252210: 250184,
 250185: 250184,
 65925: 250184,
 135980: 250184,
 251369: 250184,
 226201: 226200,
 256894: 256893,
 239034: 256893,
 215937: 256893,
 180648: 226201,
 180662: 226201,
 226202: 226201,
 16943: 250185,
 250186: 250185,
 254770: 250185,
 26806: 250185,
 125941: 250185,
 255264: 250185,
 256895: 256894,
 255064: 256894,
 226203: 226202,
 255642: 226202,
 250326: 251369,
 176607: 251369,
 254036: 251369,
 254397: 254396,
 256896: 256895,
 255208: 256895,
 226204: 226203,
 244153: 250186,
 250187: 250186,
 24630: 250186,
 254037: 254036,
 212121: 254397,
 180666: 254397,
 1210: 2

In [610]:
graph.conn_at_index(226197)

line                                  143
departure_time                   24:24:00
arrival_time                     24:27:00
start_stop                        Śliczna
end_stop          Uniwersytet Ekonomiczny
departure_sec                        1440
arrival_sec                          1620
Name: 226197, dtype: object

In [408]:
Graph(pd.read_csv(DATA_DIR / 'connection_graph.csv', 
                               usecols=['line', 'departure_time', 'arrival_time', 'start_stop', 'end_stop'], 
                               dtype=str)).get_nodes()

array(['Zajezdnia Obornicka', 'Paprotna', 'Obornicka (Wołowska)',
       'Bezpieczna', 'Bałtycka', 'Broniewskiego', 'Pola', 'Syrokomli',
       'Kasprowicza', 'pl. Daniłowskiego', 'Przybyszewskiego',
       'Czajkowskiego', 'KOSZAROWA (Uniwersytet)', 'KOSZAROWA (Szpital)',
       'Berenta', 'KROMERA', 'Mosty Warszawskie', 'Wyszyńskiego',
       'Ogród Botaniczny', 'Katedra',
       'Urząd Wojewódzki (Muzeum Narodowe)', 'Poczta Główna',
       'skwer Krasińskiego', 'Wzgórze Partyzantów', 'Renoma',
       'Arkady (Capitol)', 'Pl. Hirszfelda', 'Krucza',
       'Krucza (Mielecka)', 'Inżynierska', 'Aleja Pracy', 'FAT',
       'GRABISZYŃSKA (Cmentarz)', 'Solskiego', 'Wiejska', 'Kadłubka',
       'Stanki', 'Bukowskiego', 'RACŁAWICKA', 'Rymarska', 'Wawrzyniaka',
       'Chłodna', 'Sowia', 'KRZYKI', 'Dworzec Główny (Dworcowa)',
       'Krasińskiego', 'Damrota', 'Koszarowa', 'Sołtysowicka',
       'Poprzeczna', 'Redycka', 'Bagatela', 'SOŁTYSOWICE', 'POŚWIĘTNE',
       'Wołowska', 'Kępińska', 'Ka